In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset

In [2]:
# Load CIFAR-100
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform)

trainset = DataLoader(trainset, batch_size=64, shuffle=True)
testset = DataLoader(testset, batch_size=64, shuffle=False)


100%|██████████| 169001437/169001437 [00:01<00:00, 97099800.95it/s] 


Extracting ./data/cifar-100-python.tar.gz to ./data
Files already downloaded and verified


In [3]:
# Model
class CNN(nn.Module):
    def __init__(self):

        super(CNN, self).__init__()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        self.convolution_1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.convolution_2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.convolution_3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)

        self.linear_layer_1 = nn.Linear(128 * 4 * 4, 512)
        self.linear_layer_2 = nn.Linear(512, 100)


    def forward(self, x):
        x = self.pool(nn.functional.relu(self.convolution_1(x)))
        x = self.pool(nn.functional.relu(self.convolution_2(x)))
        x = self.pool(nn.functional.relu(self.convolution_3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = nn.functional.relu(self.linear_layer_1(x))
        x = self.linear_layer_2(x)
        return x

In [ ]:
# Initialize model
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Put model in GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Train model
epochs = 30
for epoch in range(epochs):

    model.train()

    for i, (images, labels) in enumerate(trainset):

        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    print(f'Epoch [{epoch + 1}]')
    print(f'Loss: {loss.item():.4f}')

Epoch [1]
Loss: 2.8258
Epoch [2]
Loss: 1.7600
Epoch [3]
Loss: 1.8401
Epoch [4]
Loss: 2.0146
Epoch [5]
Loss: 1.9149
Epoch [6]
Loss: 1.7809
Epoch [7]
Loss: 1.5063
Epoch [8]
Loss: 1.0980
Epoch [9]
Loss: 0.8968
Epoch [10]
Loss: 1.0871
Epoch [11]
Loss: 0.9639
Epoch [12]
Loss: 0.5508
Epoch [13]
Loss: 0.2935
Epoch [14]
Loss: 0.6133
Epoch [15]
Loss: 0.2655
Epoch [16]
Loss: 0.2447
Epoch [17]
Loss: 0.3271
Epoch [18]
Loss: 0.4064
Epoch [19]
Loss: 0.0848
Epoch [20]
Loss: 0.0452
Epoch [21]
Loss: 0.0257
Epoch [22]
Loss: 0.2009
Epoch [23]
Loss: 0.1878
Epoch [24]
Loss: 0.0428
Epoch [25]
Loss: 0.2624
Epoch [26]
Loss: 0.0056
Epoch [27]
Loss: 0.1480
Epoch [28]
Loss: 0.3767
Epoch [29]
Loss: 0.1630


In [ ]:
# Evaluate and test

accuracy = 0
model.eval()
predictions_array = []
labels_array = []

for i in range(1):
  for j, batch in enumerate(testset):

    images, labels = batch
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predictions = torch.max(outputs, 1)

    # Outputs
    predictions_array += predictions
    labels_array += labels

# Initialize
correlation_counter = 0
samples = len(predictions_array)

# Determine correlated outputs
for i in range(len(predictions_array)):
  if predictions_array[i].item() == labels_array[i].item():
    correlation_counter += 1
temporary_accuracy = correlation_counter/samples

# Print accuracy
if temporary_accuracy > accuracy:
  accuracy = tmp
  print(accuracy)